In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.feature_selection import RFECV
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# load the spam training data, split in to a training and test set
with open('spam/spam.data', 'r') as f:
    all_data = np.array([
        [float(value) for value in line.split()]
        for line in f
    ])

all_train, all_test = train_test_split(all_data, test_size=.2)

# set all data to mean 0, std 1 ( needed by some of the regression methods )
scaler = StandardScaler().fit(all_train)
all_train = scaler.transform(all_train)
all_test = scaler.transform(all_test)

loaded_train_X, loaded_train_Y = all_train[:, :-1], all_train[:, -1]
loaded_test_X, loaded_test_Y = all_test[:, :-1], all_test[:, -1]

In [3]:
def display_table(train_X, train_Y, test_X, test_Y):
    # train each of our models on the training set
    ols_model = LinearRegression(fit_intercept=False).fit(train_X, train_Y)

    lasso_model = LassoCV(cv=8, fit_intercept=False).fit(train_X, train_Y)

    ridge_model = RidgeCV(cv=8, fit_intercept=False).fit(train_X, train_Y)

    feature_select = RFECV(LinearRegression(fit_intercept=False), cv=8).fit(train_X, train_Y)

    class FeatureSelectWrapper(object):
        """
        Wraps RFECV to produce a coefficent vector with 0s at ununsed columns rather than a smaller vector
        """
        def __init__(self, real_feature_select):
            self.coef_ = []
            self._estimator = real_feature_select
            coef_index = 0
            for mask in real_feature_select.support_:
                if mask:
                    self.coef_.append(real_feature_select.estimator_.coef_[coef_index])
                    coef_index += 1
                else:
                    self.coef_.append(0)

        def predict(self, x_values):
            return self._estimator.predict(x_values)

    feature_select_wrapper = FeatureSelectWrapper(feature_select)

    # sklearn doesn't implement Principle Component Regression, so implement it ourselves
    def pcr(X, Y, k):
        assert k <= X.shape[1]
        u, s, v = np.linalg.svd(X)
        vk = v[:, :k]
        Wk = np.dot(X, vk)
        gamma_hat = np.dot(np.dot(np.linalg.inv(np.dot(Wk.T, Wk)), Wk.T), Y)
        return np.dot(vk, gamma_hat)

    class PCREstimator(BaseEstimator):
        def __init__(self, directions=1):
            self.coef_ = None
            self.directions=directions

        def fit(self, X, y):
            self.coef_ = pcr(X, y, self.directions)

        def predict(self, X):
            assert self.coef_ is not None, 'Estimator not fit'
            return np.dot(X, self.coef_)

    fit_pcr = GridSearchCV(PCREstimator(), {'directions': range(1, train_X.shape[1] + 1)}, 'r2', cv=8).fit(train_X, train_Y)

    fit_pls = GridSearchCV(PLSRegression(), {'n_components': range(1, train_X.shape[1] + 1)}, 'r2', cv=8, return_train_score=True).fit(train_X, train_Y)

    # calculate column coefficents and train/test error for each model
    all_estimators = [
        ('LS', ols_model),
        ('Best Subset', feature_select_wrapper),
        ('Ridge', ridge_model),
        ('Lasso', lasso_model),
        ('PCR', fit_pcr.best_estimator_),
        ('PLS', fit_pls.best_estimator_)
    ]

    all_coefs = np.array([
        np.reshape(estimator.coef_, test_X.shape[1])
        for _, estimator in all_estimators
    ]).T

    # set all 0 values to n/a so it appears as blank in the table
    zero_value_mask = abs(all_coefs) < .000001
    all_coefs[zero_value_mask] = np.nan

    train_errors = []
    test_errors = []
    for _, estimator in all_estimators:
        train_errors.append(mean_squared_error(train_Y, estimator.predict(train_X)))
        test_errors.append(mean_squared_error(test_Y, estimator.predict(test_X)))

    train_test_errors = pd.DataFrame(
        [train_errors, test_errors], 
        columns=[name for name, _ in all_estimators],
        index=['Train Error', 'Test Error'])

    # display the results
    return pd.DataFrame(
        all_coefs, 
        columns=[name for name, _ in all_estimators]).fillna('').append(train_test_errors)

In [4]:
# print a table matching books for spam data
display_table(loaded_train_X, loaded_train_Y, loaded_test_X, loaded_test_Y)

/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


,LS,Best Subset,Ridge,Lasso,PCR,PLS
0,-0.036110,-0.036637,-0.035866,-0.0267818,-0.033798,-0.036663
1,-0.030507,-0.0298764,-0.030408,-0.0234702,-0.046519,-0.030949
2,0.035183,0.0360065,0.035209,0.0323751,0.071214,0.035090
3,0.038179,0.0380054,0.038124,0.0341091,0.028679,0.038801
4,0.110689,0.111304,0.110466,0.107658,0.106853,0.110986
5,0.060405,0.0606352,0.060368,0.058904,0.061114,0.060327
6,0.175143,0.175378,0.174792,0.175833,0.163806,0.175843
7,0.081245,0.0819069,0.081119,0.0798151,0.087373,0.081557
8,0.059772,0.0623436,0.059723,0.05934,0.062577,0.059179
9,0.011132,,0.011173,0.0084412,0.013562,0.009840
